# Accessing ACLED Data Programmatic Regional Comparison
ACLED API documentation: https://acleddata.com/acleddatanew/wp-content/uploads/dlm_uploads/2019/01/API-User-Guide2020.pdf


#### The notebook is designed to programmatically compare two regoins of ACLED data to each other

# Info for Variables
- start_date and end_date
    - The time range you want to search on.  Formatting is YYYY-MM-DD 
- reg
    - Select two regions of interest
        - 1 - Western Africa
        - 2 - Middle Africa
        - 3 - Eastern Africa
        - 4 - Southern Africa
        - 5 - Northern Africa
        - 7 - Southern Asia
        - 9 - South-Eastern Asia
        - 11 - Middle East
        - 12 - Europe
        - 13 - Caucasus and Central Asia
        - 14 - Central America
        - 15 - South America
        - 16 - Caribbean
   
* Note - Make sure that all variables have quotes around them (example: '12')

# Populate Variables

In [1]:
##Variables
start_date = '2019-01-01'
end_date = '2019-12-31'
reg = '14'
reg1 = '15'


## Shouldn't have to change anything in the cells below

### Pulling the requested regions

In [2]:
##import libraries and packages
import requests
import pandas as pd
from pandas.io.json import json_normalize 

##
URL = "https://api.acleddata.com/region/read?terms=accept"
cnts = requests.get(url = URL)
data = cnts.json()
df_reg = json_normalize(data, 'data')
df_reg['count'] = df_reg['event_count'].astype(int)
total = df_reg['count'].sum(axis=0)
##create dictionary for region numeric code and counts of events in each region
dictionary_lim = pd.Series(df_reg.event_count.values, index = df_reg.region).to_dict()
##get total count of events worldwide 
df_reg['count'] = df_reg['event_count'].astype(int)
total = df_reg['count'].sum(axis=0)
##create dictionary for region numeric code and counts of events in each region
dictionary_lim = pd.Series(df_reg.event_count.values, index = df_reg.region).to_dict()
print(dictionary_lim)

##Query be region and save off as csv using loop of reg and reg1
regs = [reg, reg1]
a = []
for r in regs: 
    lim = dictionary_lim[r] 
    regs = r
    URL2 = 'https://api.acleddata.com/acled/read?terms=accept'
    PARAMS = "&limit="+lim+"&region="+regs+"&event_date="+start_date+"|"+end_date+"&event_date_where=BETWEEN" 
    d_req = requests.get(url = URL2, params = PARAMS)
    data_event = d_req.json()
    df_event = json_normalize(data_event, 'data')
    df_event.to_csv('Output_' + str(r) + "_"+ str(start_date)+"_"+str(end_date) + '.csv')
    print(df_event)
   

{'1': '37238', '2': '29225', '3': '69334', '4': '20951', '5': '50486', '7': '155934', '9': '34586', '11': '195500', '12': '48774', '13': '64367', '14': '21478', '15': '28147', '16': '1577'}
                                         actor1  \
0                               B-18: Barrio-18   
1                    Unidentified Gang (Mexico)   
2                    Unidentified Gang (Mexico)   
3             Unidentified Armed Group (Mexico)   
4                 El Pelon de las Playas Cartel   
5                           Protesters (Mexico)   
6             Unidentified Armed Group (Mexico)   
7             Unidentified Armed Group (Mexico)   
8                              Rioters (Mexico)   
9                       MS-13: Mara Salvatrucha   
10                   Unidentified Gang (Mexico)   
11          Unidentified Armed Group (Honduras)   
12            Unidentified Armed Group (Mexico)   
13                          Protesters (Mexico)   
14         Unidentified Armed Group (Guatemal

                                                  actor1  \
0                Unidentified Gang and/or Police Militia   
1      Military Forces of Brazil (2019-) Military Police   
2                    Unidentified Armed Group (Colombia)   
3                                 Protesters (Argentina)   
4                                     Protesters (Chile)   
5                      Unidentified Armed Group (Brazil)   
6                      Unidentified Armed Group (Brazil)   
7                Unidentified Gang and/or Police Militia   
8      Military Forces of Venezuela (1999-) DGCIM: Di...   
9      Police Forces of Venezuela (1999-) FAES: Speci...   
10                                Protesters (Venezuela)   
11                            Unidentified Gang (Brazil)   
12               Unidentified Gang and/or Police Militia   
13                   Unidentified Armed Group (Colombia)   
14     Military Forces of Brazil (2019-) Military Police   
15               Unidentified Gang and/o

In [5]:
df1 = pd.read_csv('Output_' + str(reg) + "_"+ str(start_date)+"_"+str(end_date) + '.csv')
df2 = pd.read_csv('Output_' + str(reg1) + "_"+ str(start_date)+"_"+str(end_date) + '.csv')

### Actor1 Type Comparison using inter1

#### Types of Actor
Inter Code 1: Governments and State Security Services
Inter Code 2: Rebel Groups
Inter Code 3: Political Militia
Inter Code 4: Identity Militias
Inter Code 5: Rioters
Inter Code 6: Protesters
Inter Code 7: Civilians
Inter Code 8: External/Other Forces

In [6]:
act_dict = {1: 'Governments and State Security Services', 2:'Rebel Groups', 3: 'Political Militia', 4: 'Identity Militias', 5: 'Rioters', 6: 'Protestors', 7: 'Civilians', 8: 'External Forces/Other Forces'}
act1_cnt = df1.groupby('inter1').size().reset_index()
act1_cnt['actor_type1'] = act1_cnt['inter1'].map(act_dict)
act1b_cnt = df2.groupby('inter1').size().reset_index()
act1b_cnt['actor_type1'] = act1b_cnt['inter1'].map(act_dict)
act1_cnt.rename(columns={act1_cnt.columns[1]: 'reg'+str(reg)+'_cnt'}, inplace = True)
act1b_cnt.rename(columns={act1b_cnt.columns[1]: 'reg'+str(reg1)+'_cnt'}, inplace = True)
act1b_cnt = act1b_cnt.drop(columns= 'inter1')
act1b_cnt
act1_tot = pd.merge(act1_cnt, act1b_cnt, on='actor_type1')
act1_tot.to_csv('act1_comparison.csv')
act1_tot

,inter1,reg14_cnt,actor_type1,reg15_cnt
0,1,768,Governments and State Security Services,2348
1,3,8971,Political Militia,6473
2,4,82,Identity Militias,115
3,5,1198,Rioters,2598
4,6,6263,Protestors,10903
5,7,8,Civilians,27
6,8,9,External Forces/Other Forces,26


### Actor2 Type Comparison using inter2

#### Types of Actor
Inter Code 1: Governments and State Security Services
Inter Code 2: Rebel Groups
Inter Code 3: Political Militia
Inter Code 4: Identity Militias
Inter Code 5: Rioters
Inter Code 6: Protesters
Inter Code 7: Civilians
Inter Code 8: External/Other Forces

In [ ]:
act_dict = {1: 'Governments and State Security Services', 2:'Rebel Groups', 3: 'Political Militia', 4: 'Identity Militias', 5: 'Rioters', 6: 'Protestors', 7: 'Civilians', 8: 'External Forces/Other Forces'}
act2_cnt = df1.groupby('inter2').size().reset_index()
act2_cnt['actor_type2'] = act2_cnt['inter2'].map(act_dict)
act2b_cnt = df2.groupby('inter2').size().reset_index()
act2b_cnt['actor_type2'] = act2b_cnt['inter2'].map(act_dict)
act2_cnt.rename(columns={act2_cnt.columns[1]: 'reg'+str(reg)+'_cnt'}, inplace = True)
act2b_cnt.rename(columns={act2b_cnt.columns[1]: 'reg'+str(reg1)+'_cnt'}, inplace = True)
act2b_cnt = act2b_cnt.drop(columns= 'inter2')
act2b_cnt
act2_tot = pd.merge(act2_cnt, act2b_cnt, on='actor_type2')
act2_tot.to_csv('act2_comparison.csv')
act2_tot